In [16]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [17]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [18]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [19]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [20]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [21]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [22]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [23]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [24]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [25]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [26]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [27]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [28]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [30]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.7840402e-03  3.9811170e-04  4.6922537e-03 ...  1.6863242e-03
    1.9016602e-03 -2.5081495e-03]
  [-2.2132553e-03 -1.2324499e-03  4.6493853e-03 ... -6.6064805e-04
   -4.0373984e-03 -7.5725662e-03]
  [-4.7531356e-03 -2.6957481e-03  7.0694955e-03 ... -3.0486973e-04
   -3.7577529e-03 -5.7087634e-03]
  ...
  [-1.1365549e-03 -3.5946439e-03 -2.8223684e-04 ...  2.6657023e-03
   -1.5238433e-03 -7.4529708e-03]
  [ 2.4522338e-03  1.7424782e-03 -1.1420511e-03 ...  4.0047122e-03
   -1.1666918e-02 -8.0910726e-03]
  [ 3.8113052e-03 -1.0789669e-04  6.0638739e-04 ...  5.2251499e-03
   -7.7445796e-03 -4.9998062e-03]]

 [[-3.7840402e-03  3.9811170e-04  4.6922537e-03 ...  1.6863242e-03
    1.9016602e-03 -2.5081495e-03]
  [-6.4240494e-03 -8.3850464e-05  9.3205459e-03 ...  7.4986843e-03
   -9.3386543e-04  2.0248946e-03]
  [-7.2030062e-03 -2.6411419e-03  4.4001020e-03 ...  9.3168253e-03
   -3.8666972e-03  5.7250066e-03]
  ...
  [ 1.7452647e-04  3.6327704e-03 -1.5370895e-03 ...  1.5499992e

In [31]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00378404  0.00039811  0.00469225 ...  0.00168632  0.00190166
  -0.00250815]
 [-0.00221326 -0.00123245  0.00464939 ... -0.00066065 -0.0040374
  -0.00757257]
 [-0.00475314 -0.00269575  0.0070695  ... -0.00030487 -0.00375775
  -0.00570876]
 ...
 [-0.00113655 -0.00359464 -0.00028224 ...  0.0026657  -0.00152384
  -0.00745297]
 [ 0.00245223  0.00174248 -0.00114205 ...  0.00400471 -0.01166692
  -0.00809107]
 [ 0.00381131 -0.0001079   0.00060639 ...  0.00522515 -0.00774458
  -0.00499981]], shape=(100, 65), dtype=float32)


In [32]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-3.7840402e-03  3.9811170e-04  4.6922537e-03  1.9496015e-03
  6.4365012e-03  3.0190521e-04  2.9715925e-04  3.0660308e-03
  3.9518671e-03  1.8749833e-03  1.9632181e-04  4.7880542e-03
  1.7771299e-03  1.4443602e-03 -7.8288314e-04  7.7698461e-04
 -3.5456691e-03  7.8186579e-04 -1.0324561e-03 -7.5947233e-03
  8.9033449e-05 -3.2564406e-03  8.9618651e-04  2.7344113e-03
  1.6594427e-03  5.0389383e-05  2.0181376e-03 -5.9040822e-04
 -7.0868768e-03  1.1465284e-03 -2.0128724e-03 -3.3764599e-04
 -4.4346703e-03 -2.4349757e-03 -6.7496430e-03  6.5233456e-03
  6.1590219e-04 -1.2890297e-03 -9.4302453e-04  4.5214728e-03
 -6.5005622e-03  4.0911939e-03  4.3505877e-03 -2.1388321e-03
 -1.0934658e-05 -3.4409942e-04 -2.1179225e-03  1.5123256e-03
 -5.6290817e-03  5.7008432e-04 -2.0968835e-03  8.4166892e-04
 -7.2987308e-04 -4.1485131e-03 -2.3748407e-03  1.1423983e-03
  2.7725578e-03 -3.9304688e-04  4.9538813e-03  2.4709017e-03
  1.3237154e-03  3.7867455e-03  1.6863242e-03  1.9016602e-03
 -2.508149

In [33]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

".W bbb?!YIp:Of-zYPz.XrZBf3fJi, wj' &PZM,nnDz,LKuXzV?lHeo;ldZblIbBKizOIHK$ujYZp;Zptxbf&lXTe.at PfmuTe"

In [34]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [35]:
model.compile(optimizer='adam', loss=loss)

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 518s 3s/step - loss: 2.5773
Epoch 2/50
172/172 [==============================] - 575s 3s/step - loss: 1.8856
Epoch 3/50
152/172 [=========================>....] - ETA: 1:06 - loss: 1.6495

KeyboardInterrupt: 

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))